In [2]:
from ppq import *
from ppq.api import *
import torch
import cfg


      ____  ____  __   ____                    __              __
     / __ \/ __ \/ /  / __ \__  ______ _____  / /_____  ____  / /
    / /_/ / /_/ / /  / / / / / / / __ `/ __ \/ __/ __ \/ __ \/ /
   / ____/ ____/ /__/ /_/ / /_/ / /_/ / / / / /_/ /_/ / /_/ / /
  /_/   /_/   /_____\___\_\__,_/\__,_/_/ /_/\__/\____/\____/_/




In [ ]:
# 消除冗余算子
import onnx
from onnxsim import simplify

onnx_model = onnx.load(f'{os.path.join(cfg.FP32_BASE_PATH, name)}-FP32.onnx') 
model_simp, check = simplify(onnx_model)   #对onnx模型进行简化，消除冗余算子        
assert check, "Simplified ONNX model could not be validated"
onnx.save(model_simp, f'{os.path.join(cfg.FP32_BASE_PATH, name)}-FP32.onnx')

In [5]:
model_path = "/home/geng/tinyml/ppq/benchmark/detection/FP32_model/Retinanet-FP32-end-11.onnx"
# model_path = "/home/geng/tinyml/ppq/benchmark/detection/retinanet.onnx"
input_size = [3,480,640]
dataloader = [torch.rand(size=input_size) for _ in range(32)]
step = 32
config = cfg.PLATFORM_CONFIGS["TRT"]

In [6]:
ppq_quant_ir = quantize_onnx_model(
    onnx_import_file=model_path, calib_dataloader=dataloader, calib_steps=step, 
    setting=config["QuanSetting"],input_shape=input_size, collate_fn=lambda x: x.to(cfg.DEVICE), 
    platform=config["QuantPlatform"], do_quantize=True)

opset version is not supported, can not generate dispatching scheme with op Less_293(Less), currently we support only [(12, 16)], however 11 was given.
opset version is not supported, can not generate dispatching scheme with op Less_357(Less), currently we support only [(12, 16)], however 11 was given.
opset version is not supported, can not generate dispatching scheme with op Less_421(Less), currently we support only [(12, 16)], however 11 was given.
opset version is not supported, can not generate dispatching scheme with op Less_485(Less), currently we support only [(12, 16)], however 11 was given.
opset version is not supported, can not generate dispatching scheme with op Less_549(Less), currently we support only [(12, 16)], however 11 was given.
opset version is not supported, can not generate dispatching scheme with op Less_691(Less), currently we support only [(12, 16)], however 11 was given.
opset version is not supported, can not generate dispatching scheme with op Less_737(Les

RuntimeError: Error happens when dealing with operation Identity_0(TargetPlatform.SHAPE_OR_INDEX) - inputs:['onnx::Clip_1463_0'], outputs:['onnx::Clip_1467']

In [ ]:
"""
op Less need opset (12,16)
op TopK need opset (1,12)
Upsample not supported
"""